In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader, TensorDataset 

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

import numpy as np 
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), 
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

train_ds = datasets.MNIST('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
valid_ds = datasets.MNIST('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)
train_dl = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

In [ ]:
plt.imshow(images[0].numpy().squeeze(), cmap='gray_r')

In [ ]:
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28).to(device), y.to(device)

In [ ]:
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs),
    )

In [ ]:
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

train_dl, valid_dl = get_data(train_ds, valid_ds, 64)
train_dl = WrappedDataLoader(train_dl, preprocess) #now train/valid_dl count as wrappeddataloader objects 
valid_dl = WrappedDataLoader(valid_dl, preprocess)

In [ ]:
model.to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
from time import time

optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time0 = time()
epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in train_dl:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(train_dl)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

In [ ]:
torch.save(model, './cuda_wrap.pt') 